# EE 451: Communications Systems
## Lesson 2 - Signal Analysis Fundamentals

### Learning Objectives
By the end of this lesson, you will be able to:
- Apply convolution to analyze linear systems with arbitrary input signals
- Distinguish between energy signals and power signals, and calculate energy/power
- Represent sinusoidal signals as rotating phasors in the complex plane
- Explain the concept and utility of analytic signals in communications
- Use impulse and step functions to characterize system behavior

### Textbook Reference
Haykin & Moher, Chapter 2.1

In [ ]:
# Setup: Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, fftfreq
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')

# Matplotlib settings for publication-quality plots
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

print("Setup complete! NumPy version:", np.__version__)
print("Matplotlib backend:", plt.get_backend())

## 1. Impulse and Step Functions

### Unit Impulse δ(t)
The unit impulse (Dirac delta function) is a fundamental signal in communications:

**Informal definition:**
- $\delta(t) = 0$ for $t \neq 0$
- $\int_{-\infty}^{\infty} \delta(t) dt = 1$

**Sifting property:**
$$\int_{-\infty}^{\infty} f(t)\delta(t - t_0) dt = f(t_0)$$

Physical interpretation: Idealized "click" or "tap" - no true physical impulse exists, but approximations are useful.

### Unit Step u(t)
$$u(t) = \begin{cases} 0 & t < 0 \\ 1 & t \geq 0 \end{cases}$$

Relationship to impulse: $\delta(t) = \frac{du(t)}{dt}$

In [ ]:
# Visualize impulse and step functions
t = np.linspace(-2, 5, 1000)

# Step function
u = np.heaviside(t, 1)

# Approximate impulse (very narrow pulse)
delta_approx = np.zeros_like(t)
impulse_idx = np.argmin(np.abs(t))  # Find index closest to t=0
delta_approx[impulse_idx] = 10  # Height scaled for visibility

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot step function
ax1.plot(t, u, 'b-', linewidth=2.5)
ax1.axhline(y=0, color='k', linewidth=0.5)
ax1.axvline(x=0, color='k', linewidth=0.5)
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Amplitude')
ax1.set_title('Unit Step Function: $u(t)$')
ax1.grid(True, alpha=0.3)
ax1.set_ylim(-0.5, 1.5)

# Plot impulse (approximation)
ax2.stem([0], [1], basefmt=' ', linefmt='r-', markerfmt='ro')
ax2.axhline(y=0, color='k', linewidth=0.5)
ax2.axvline(x=0, color='k', linewidth=0.5)
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Amplitude')
ax2.set_title('Unit Impulse Function: $\\delta(t)$')
ax2.grid(True, alpha=0.3)
ax2.set_xlim(-2, 5)
ax2.set_ylim(-0.2, 1.5)

plt.tight_layout()
plt.show()

print("Step function: Switches from 0 to 1 at t=0")
print("Impulse function: Infinite height at t=0, zero elsewhere, but area = 1")

In [ ]:
# Worked Example 1: Sifting property
# Evaluate ∫ cos(2πt) δ(t - 0.5) dt from -∞ to ∞

t0 = 0.5  # Impulse location
f_t0 = np.cos(2 * np.pi * t0)  # Evaluate f(t) at t = t0

print("Worked Example 1: Sifting Property")
print("="*50)
print(f"Evaluate: ∫ cos(2πt) δ(t - 0.5) dt")
print(f"\nUsing sifting property: f(t₀) where t₀ = {t0}")
print(f"Result: cos(2π × {t0}) = cos(π) = {f_t0:.4f}")
print(f"\nThe impulse 'picks out' the value of the function at t = {t0}")

## 2. Convolution

Convolution is fundamental to understanding how signals pass through systems.

**Convolution integral:**
$$y(t) = x(t) * h(t) = \int_{-\infty}^{\infty} x(\tau)h(t - \tau) d\tau$$

where:
- $x(t)$ is the input signal
- $h(t)$ is the system impulse response
- $y(t)$ is the output signal

**Physical interpretation:** The output is the sum of scaled and shifted impulse responses.

**Properties:**
- Commutative: $x * h = h * x$
- Associative: $(x * h_1) * h_2 = x * (h_1 * h_2)$
- Distributive: $x * (h_1 + h_2) = x * h_1 + x * h_2$
- Identity: $x * \delta = x$

In [ ]:
# Worked Example 2: Rectangular pulse through RC lowpass filter
# Input: x(t) = u(t) - u(t - T) (pulse of width T)
# Impulse response: h(t) = (1/RC)e^(-t/RC) u(t)

# Parameters
T = 1.0  # Pulse width (seconds)
RC = 0.5  # Time constant (seconds)
t = np.linspace(-0.5, 3, 1000)

# Input pulse
x = np.where((t >= 0) & (t < T), 1, 0)

# Impulse response
h = np.where(t >= 0, (1/RC) * np.exp(-t/RC), 0)

# Convolution using numpy
dt = t[1] - t[0]
y_conv = np.convolve(x, h, mode='same') * dt

# Theoretical output (derived)
y_theory = np.zeros_like(t)
for i, t_val in enumerate(t):
    if 0 < t_val < T:
        y_theory[i] = 1 - np.exp(-t_val/RC)
    elif t_val >= T:
        y_theory[i] = (np.exp((T-t_val)/RC) - np.exp(-t_val/RC))

# Plot
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10))

# Input signal
ax1.plot(t, x, 'b-', linewidth=2.5, label='Input: $x(t) = u(t) - u(t-T)$')
ax1.set_ylabel('Amplitude')
ax1.set_title('Input Signal: Rectangular Pulse')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_xlim(-0.5, 3)

# Impulse response
ax2.plot(t, h, 'g-', linewidth=2.5, label='$h(t) = \\frac{1}{RC}e^{-t/RC}u(t)$')
ax2.set_ylabel('Amplitude')
ax2.set_title(f'System Impulse Response (RC = {RC} s)')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_xlim(-0.5, 3)

# Output (convolution)
ax3.plot(t, y_conv, 'r-', linewidth=2.5, label='Output: $y(t) = x(t) * h(t)$')
ax3.plot(t, y_theory, 'k--', linewidth=1.5, alpha=0.7, label='Theoretical')
ax3.set_xlabel('Time (s)')
ax3.set_ylabel('Amplitude')
ax3.set_title('Output Signal (Convolution Result)')
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.set_xlim(-0.5, 3)

plt.tight_layout()
plt.show()

print("Key Observation: Convolution causes pulse spreading")
print(f"  - Rise time: ~{3*RC:.2f} s (3×RC)")
print(f"  - This is a preview of Intersymbol Interference (ISI) in digital communications")

## 3. Energy and Power Signals

### Energy Signals
A signal with finite energy:
$$E = \int_{-\infty}^{\infty} |x(t)|^2 dt < \infty$$

**Examples:**
- Rectangular pulse: $E = A^2T$ (where $A$ = amplitude, $T$ = duration)
- Exponential decay: $e^{-at}u(t)$ has $E = 1/(2a)$

### Power Signals
Average power over infinite time:
$$P = \lim_{T\to\infty} \frac{1}{T} \int_{-T/2}^{T/2} |x(t)|^2 dt$$

**Examples:**
- Sinusoid: $A \cos(2\pi ft)$ has $P = A^2/2$
- DC signal: $P = A^2$

**Key distinction:** A signal cannot have both finite energy AND finite non-zero power.
- Energy signals: Transient, finite duration (e.g., pulses in digital comm)
- Power signals: Continuous, often periodic (e.g., carriers, noise)

In [ ]:
# Worked Example 3: Calculate energy and power for rectangular pulse
A = 2.0  # Amplitude (volts)
T = 1.0  # Duration (seconds)

# Energy calculation
E = A**2 * T

# Power calculation (limit as observation time → ∞)
# For finite duration pulse, average power → 0

print("Worked Example 3: Rectangular Pulse Energy/Power")
print("="*60)
print(f"Signal: x(t) = {A} V for 0 ≤ t < {T} s, zero elsewhere")
print(f"\nEnergy: E = A²T = {A}² × {T} = {E:.2f} J (Joules)")
print(f"\nAverage Power: P = lim(T'→∞) E/T' = {E}/∞ = 0 W")
print(f"\nConclusion: This is an ENERGY signal (finite energy, zero average power)")

# Example: Sinusoid
print("\n" + "="*60)
print("Example: Sinusoid x(t) = A cos(2πft)")
A_sin = 1.0
P_sin = A_sin**2 / 2
print(f"\nAmplitude: A = {A_sin} V")
print(f"Average Power: P = A²/2 = {P_sin:.2f} W")
print(f"Energy: E = ∫|x(t)|² dt = ∞ (infinite)")
print(f"\nConclusion: This is a POWER signal (finite power, infinite energy)")

In [ ]:
# Visualize energy vs. power signals
t = np.linspace(-2, 8, 1000)

# Energy signal: Exponential decay
a = 1.0
x_energy = np.exp(-a*t) * np.heaviside(t, 1)

# Power signal: Sinusoid
f = 1.0
x_power = np.cos(2*np.pi*f*t)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Energy signal
ax1.plot(t, x_energy, 'b-', linewidth=2.5)
ax1.fill_between(t, 0, x_energy**2, alpha=0.3, label='$|x(t)|^2$ (energy density)')
ax1.set_ylabel('Amplitude')
ax1.set_title('Energy Signal: $x(t) = e^{-t}u(t)$ (finite energy, zero average power)')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_xlim(-2, 8)

# Power signal
ax2.plot(t, x_power, 'r-', linewidth=2.5)
ax2.axhline(y=np.sqrt(0.5), color='g', linestyle='--', linewidth=2, label='$\\sqrt{P}$ (RMS value)')
ax2.axhline(y=-np.sqrt(0.5), color='g', linestyle='--', linewidth=2)
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Amplitude')
ax2.set_title('Power Signal: $x(t) = \cos(2\pi ft)$ (finite power, infinite energy)')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_xlim(-2, 8)

plt.tight_layout()
plt.show()

## 4. Phasor Representation (Review from Lesson 1)

A general sinusoid:
$$x(t) = A \cos(2\pi ft + \phi)$$

Can be represented using complex exponentials:
$$x(t) = \text{Re}\{A e^{j\phi} e^{j2\pi ft}\} = \text{Re}\{\tilde{X} e^{j2\pi ft}\}$$

where $\tilde{X} = A e^{j\phi}$ is the **phasor** (time-independent complex number).

**Adding sinusoids at the same frequency:**
$$x_1(t) + x_2(t) = \text{Re}\{(\tilde{X}_1 + \tilde{X}_2) e^{j\omega t}\}$$

Simply add phasors, then convert back to time domain!

In [ ]:
# Worked Example 4: Add cos(ωt) + cos(ωt + 90°)
A1 = 1
phi1 = 0
A2 = 1
phi2 = np.pi/2  # 90 degrees

# Phasor addition
phasor1 = A1 * np.exp(1j * phi1)
phasor2 = A2 * np.exp(1j * phi2)
phasor_sum = phasor1 + phasor2

A_result = abs(phasor_sum)
phi_result = np.angle(phasor_sum)

print("Worked Example 4: Phasor Addition")
print("="*60)
print(f"Phasor 1: {A1}∠{np.degrees(phi1):.0f}° = {phasor1}")
print(f"Phasor 2: {A2}∠{np.degrees(phi2):.0f}° = {phasor2}")
print(f"\nSum: {A_result:.4f}∠{np.degrees(phi_result):.1f}° = {phasor_sum}")
print(f"\nResult: {A_result:.4f} cos(ωt + {np.degrees(phi_result):.1f}°)")

# Verify in time domain
t = np.linspace(0, 2, 1000)
omega = 2 * np.pi * 1  # 1 Hz
x1 = A1 * np.cos(omega*t + phi1)
x2 = A2 * np.cos(omega*t + phi2)
x_sum = A_result * np.cos(omega*t + phi_result)
x_sum_direct = x1 + x2

plt.figure(figsize=(12, 6))
plt.plot(t, x1, 'b-', linewidth=2, alpha=0.6, label='$x_1(t) = \cos(\omega t)$')
plt.plot(t, x2, 'g-', linewidth=2, alpha=0.6, label='$x_2(t) = \cos(\omega t + 90°)$')
plt.plot(t, x_sum, 'r-', linewidth=3, label=f'$x(t) = {A_result:.2f}\cos(\omega t + {np.degrees(phi_result):.0f}°)$')
plt.plot(t, x_sum_direct, 'k--', linewidth=1.5, alpha=0.5, label='Direct sum (verification)')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('Phasor Addition: cos(ωt) + cos(ωt + 90°)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nVerification: Phasor method matches direct addition ✓")

## 5. Analytic Signals and Hilbert Transform

**Motivation:** Real-valued signals have symmetric spectra (positive and negative frequencies). Negative frequencies are redundant!

**Analytic signal:** Complex signal with only positive frequencies.

For real signal $x(t)$:
$$x_+(t) = x(t) + j\hat{x}(t)$$

where $\hat{x}(t)$ is the **Hilbert transform** of $x(t)$.

**Spectrum:** $X_+(f) = 2X(f)$ for $f > 0$, and $0$ for $f < 0$

**Example:** Analytic signal of cosine
- $x(t) = \cos(\omega t)$
- $\hat{x}(t) = \sin(\omega t)$ (Hilbert transform)
- $x_+(t) = \cos(\omega t) + j \sin(\omega t) = e^{j\omega t}$

**Why it matters:**
- Elegant representation of bandpass signals
- Foundation for complex baseband representation (used in SDR)
- Critical for Single Sideband (SSB) modulation

In [ ]:
# Demonstrate analytic signal for cosine
t = np.linspace(0, 2, 1000)
f = 2  # Hz
omega = 2 * np.pi * f

# Real signal
x = np.cos(omega * t)

# Hilbert transform (for cosine, it's sine)
x_hat = np.sin(omega * t)

# Analytic signal
x_analytic = x + 1j * x_hat

# Using scipy's Hilbert transform
from scipy.signal import hilbert
x_analytic_scipy = hilbert(x)

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10))

# Real signal
ax1.plot(t, x, 'b-', linewidth=2.5)
ax1.set_ylabel('Amplitude')
ax1.set_title('Real Signal: $x(t) = \cos(\omega t)$')
ax1.grid(True, alpha=0.3)

# Hilbert transform
ax2.plot(t, x_hat, 'g-', linewidth=2.5, label='Theoretical: $\sin(\omega t)$')
ax2.plot(t, x_analytic_scipy.imag, 'r--', linewidth=1.5, alpha=0.7, label='SciPy Hilbert')
ax2.set_ylabel('Amplitude')
ax2.set_title('Hilbert Transform: $\hat{x}(t)$ (90° phase shift)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Analytic signal magnitude and phase
ax3_twin = ax3.twinx()
ax3.plot(t, np.abs(x_analytic), 'r-', linewidth=2.5, label='Magnitude')
ax3_twin.plot(t, np.angle(x_analytic), 'b-', linewidth=2.5, alpha=0.7, label='Phase (rad)')
ax3.set_xlabel('Time (s)')
ax3.set_ylabel('Magnitude', color='r')
ax3_twin.set_ylabel('Phase (radians)', color='b')
ax3.set_title('Analytic Signal: $x_+(t) = e^{j\omega t}$')
ax3.legend(loc='upper left')
ax3_twin.legend(loc='upper right')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key Observations:")
print("  - Hilbert transform shifts phase by -90° (cosine → sine)")
print("  - Analytic signal has constant magnitude (= 1 for unit cosine)")
print("  - Phase increases linearly (rotating phasor)")
print("  - Contains only positive frequency component")

## 6. Practice Problems

Try these exercises to reinforce your understanding:

1. **Convolution:** What is $\delta(t-2) * u(t)$?
2. **Energy:** Calculate energy of $x(t) = 3 \cdot \text{rect}(t/4)$ (rectangular pulse, amplitude 3, width 4)
3. **Power:** What is average power of $x(t) = 2\cos(2\pi \cdot 50t) + \sin(2\pi \cdot 100t)$?
4. **Phasor:** Add $3\angle 30° + 4\angle 120°$

In [ ]:
# Solution space for practice problems
# Try solving on your own first!

print("Practice Problem Solutions")
print("="*60)

# Problem 1: δ(t-2) * u(t)
print("\n1. Convolution: δ(t-2) * u(t)")
print("   Using shift property: δ(t-2) * u(t) = u(t-2)")
print("   Result: Step function shifted right by 2 seconds")

# Problem 2: Energy of rectangular pulse
A = 3
T = 4
E = A**2 * T
print(f"\n2. Energy: x(t) = 3·rect(t/4)")
print(f"   E = A²T = {A}² × {T} = {E} J")

# Problem 3: Power of sum of sinusoids
A1 = 2
A2 = 1
P1 = A1**2 / 2
P2 = A2**2 / 2
P_total = P1 + P2  # Independent frequencies, powers add
print(f"\n3. Power: x(t) = 2cos(2π·50t) + sin(2π·100t)")
print(f"   P₁ = (2)²/2 = {P1} W")
print(f"   P₂ = (1)²/2 = {P2} W")
print(f"   P_total = {P1} + {P2} = {P_total} W")

# Problem 4: Phasor addition
p1 = 3 * np.exp(1j * np.radians(30))
p2 = 4 * np.exp(1j * np.radians(120))
p_sum = p1 + p2
print(f"\n4. Phasor: 3∠30° + 4∠120°")
print(f"   3∠30° = {p1:.4f}")
print(f"   4∠120° = {p2:.4f}")
print(f"   Sum = {abs(p_sum):.4f}∠{np.angle(p_sum, deg=True):.2f}°")

## Key Concepts Summary

1. **Impulse δ(t) and step u(t)** are fundamental building blocks for signal analysis
2. **Convolution** describes system response: $y = x * h$
   - Causes pulse spreading (preview of ISI)
3. **Energy signals** (finite E) vs. **power signals** (finite P)
   - Energy: Transient pulses (digital comm)
   - Power: Continuous signals (carriers, noise)
4. **Phasors** simplify sinusoid arithmetic at same frequency
5. **Analytic signals** eliminate negative frequency redundancy
   - Foundation for complex baseband representation
   - Critical for SSB modulation

## Next Lesson Preview
Lesson 3: Fourier Analysis Essentials
- Fourier series for periodic signals
- Fourier transforms for aperiodic signals
- Spectral visualization
- Python Lab: Hands-on Fourier analysis